# Accuinsight+ Modeler AutoDL tutorial
## 스팸 계정과 정상 계정을 분류하는 MLP 모델  
## 목차
1. [분석 개요](#mlp_분석개요)
2. [데이터](#mlp_데이터)
3. [MLP 모델 생성 및 학습](#mlp_모델_구성_및_학습)
4. [Modeler AutoDL 사용방법](#AutoDL_사용방법)

<a id='mlp_분석개요'></a>
## 분석 개요
- 본 예제에서는 kaggle에서 제공하는 [인스타그램 데이터](https://www.kaggle.com/free4ever1/instagram-fake-spammer-genuine-accounts?select=train.csv)를 사용해 스팸 계정과 정상 계정을 분류하는 MLP 모델을 만들고,  
  Accuinsight+ Modeler의 AutoDL 기능을 사용하여 하이퍼파라미터를 최적화하는 과정을 설명합니다.([이미지 출처](https://www.intechopen.com/books/artificial-neural-networks-architectures-and-applications/mlp-and-anfis-applied-to-the-prediction-of-hole-diameters-in-the-drilling-process/))
<p align="center">
    <img src="pic/MLP.jpg" height="400px" width="500px">
</p>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import easydict
import csv
from datetime import datetime, timezone

from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers, Input

<a id='mlp_데이터'></a>
## 데이터
- 설명 변수는 총 11개로, 4개의 이진 변수를 포함하고 있습니다. 
  - 이진 변수: `profile pic`, `name==username`, `external URL`, `private`
- 데이터의 클래스를 의미하는 변수는 `fake`이고 `fake`의 값이 1이면 스팸 계정이고 0이면 정상 계정입니다.

train 데이터를 불러옵니다.

In [2]:
train = pd.read_csv("data/instagram_train.csv")
train.head()

,profile pic,nums/length username,fullname words,nums/length fullname,name==username,description length,external URL,private,#posts,#followers,#follows,fake
0,1,0.27,0,0.0,0,53,0,0,32,1000,955,0
1,1,0.00,2,0.0,0,44,0,0,286,2740,533,0
2,1,0.10,2,0.0,0,0,0,1,13,159,98,0
3,1,0.00,1,0.0,0,82,0,0,679,414,651,0
4,1,0.00,2,0.0,0,0,0,1,6,151,126,0


In [3]:
train['profile pic'] = train['profile pic'].astype('category')
train['name==username'] = train['name==username'].astype('category')
train['external URL'] = train['external URL'].astype('category')
train['private'] = train['private'].astype('category')

train_X = pd.get_dummies(train)
train_Y = train_X.pop("fake")

validation 데이터를 불러옵니다.

In [4]:
valid = pd.read_csv("data/instagram_valid.csv")
valid.head()

,profile pic,nums/length username,fullname words,nums/length fullname,name==username,description length,external URL,private,#posts,#followers,#follows,fake
0,1,0.33,1,0.33,1,30,0,1,35,488,604,0
1,1,0.00,5,0.00,0,64,0,1,3,35,6,0
2,1,0.00,2,0.00,0,82,0,1,319,328,668,0
3,1,0.00,1,0.00,0,143,0,1,273,14890,7369,0
4,1,0.50,1,0.00,0,76,0,1,6,225,356,0


In [5]:
valid['profile pic'] = valid['profile pic'].astype('category')
valid['name==username'] = valid['name==username'].astype('category')
valid['external URL'] = valid['external URL'].astype('category')
valid['private'] = valid['private'].astype('category')

valid_X = pd.get_dummies(valid)
valid_Y = valid_X.pop("fake")

데이터 개수를 확인합니다.

In [6]:
print("총 데이터 개수: ", len(train) + len(valid))
print("train 데이터 개수: ", len(train))
print("validation 데이터 개수: ", len(valid))

총 데이터 개수:  696
train 데이터 개수:  576
validation 데이터 개수:  120


<a id='mlp_모델_구성_및_학습'></a>
## MLP 모델 생성 및 학습
- 본 예제에서는 모델 학습과 관련된 하이퍼파라미터인 batch size, learning_rate와  
  모델 구조를 결정하는 하이퍼파라미터인 num_nodes와 num_layers를 최적화합니다.  
  - `batch_size`: 모델 학습의 반복 한 회당 사용되는 샘플의 개수
  - `learning_rate`: 경사하강법을 통해 모델을 학습시키는 데 사용되는 스칼라값으로, 각 반복에서 경사에 곱해지는 값
  - `num_nodes`: hidden layer의 노드 개수
  - `num_layers`: hidden layer의 개수
- AutoDL을 사용하기 위해서는 __argparse__를 통해 하이퍼파라미터를 설정해야 하는데,  
  jupyter notebook에서는 argparse 사용이 불가능하기 때문에 __easydict__를 통해 하이퍼파라미터를 설정합니다.  
  실제로 AutoDL에 사용되는 [autodl_mlp_user_classification.py](autodl_mlp_user_classification.py)에서는 argparse를 통해 하이퍼파라미터를 설정합니다.

In [7]:
args = easydict.EasyDict({
    "epochs": 50,
    "batch_size": 64,
    "learning_rate": 0.0001,
    "activation_func": 'relu',
    "num_nodes": 16,
    "num_layers": 2})

MLP 모델을 생성합니다.

In [10]:
model = tf.keras.Sequential()
model.add(Input(shape=(15,)))
for i in range(args.num_layers):
    model.add(Dense(args.num_nodes, activation=args.activation_func))
model.add(Dense(2, activation=args.activation_func))

# Adam optimizer를 사용합니다.
opt = optimizers.Adam(lr=args.learning_rate, amsgrad=True)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 16)                256       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 34        
Total params: 562
Trainable params: 562
Non-trainable params: 0
_________________________________________________________________


<a id='평가지표_출력'></a>
#### 평가지표 출력
- AutoDL에서 평가 지표를 수집할 수 있도록 epoch이 종료될 때마다 평가 지표(Accuracy)를 출력합니다.
- 이번 예제에서는 __평가 지표명=평가 지표값__ 형식으로 평가 지표를 출력하는데,  
  평가 지표 설정 방법은 [Modeler AutoDL 사용방법](#평가지표_수집) 챕터에서 좀 더 자세히 다룰 것입니다.

In [12]:
class MetricHistory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print("\nEpoch {}".format(epoch + 1))
        print("Train-acc={:.4f}".format(logs['accuracy']))
        print("Validation-acc={:.4f}".format(logs['val_accuracy']))

history = MetricHistory()

<a id='튜닝_전_결과'></a>
#### 모델 학습
- 학습 결과 validation accuracy는 __65%__입니다.
- 다음 챕터인 [Modeler AutoDL 사용방법](#AutoDL_사용방법) 챕터에서 AutoDL 기능으로 MLP 모델의 하이퍼파라미터를 튜닝하여 모델 성능을 향상시키는 과정을 살펴볼 것입니다.

In [11]:
model.fit(train_X.values, train_Y.values,
          epochs=args.epochs,
          batch_size=args.batch_size,
          validation_data=(valid_X, valid_Y),
          callbacks=[history])

Epoch 1/50
1/9 [==>...........................] - ETA: 0s - loss: 8.2676 - accuracy: 0.6094
Epoch 1
Train-acc=0.6215
Validation-acc=0.6250
9/9 [==============================] - 0s 10ms/step - loss: 7.8516 - accuracy: 0.6215 - val_loss: 7.9050 - val_accuracy: 0.6250
Epoch 2/50
1/9 [==>...........................] - ETA: 0s - loss: 7.1645 - accuracy: 0.6094
Epoch 2
Train-acc=0.6250
Validation-acc=0.6417
9/9 [==============================] - 0s 4ms/step - loss: 7.6021 - accuracy: 0.6250 - val_loss: 7.7264 - val_accuracy: 0.6417
Epoch 3/50
1/9 [==>...........................] - ETA: 0s - loss: 6.8916 - accuracy: 0.6406
Epoch 3
Train-acc=0.6285
Validation-acc=0.6417
9/9 [==============================] - 0s 5ms/step - loss: 7.5310 - accuracy: 0.6285 - val_loss: 7.6428 - val_accuracy: 0.6417
Epoch 4/50
1/9 [==>...........................] - ETA: 0s - loss: 7.0932 - accuracy: 0.6406
Epoch 4
Train-acc=0.6302
Validation-acc=0.6417
9/9 [==============================] - 0s 5ms/step - loss: 7.4

Epoch 32/50
1/9 [==>...........................] - ETA: 0s - loss: 7.4228 - accuracy: 0.5781
Epoch 32
Train-acc=0.6337
Validation-acc=0.6417
9/9 [==============================] - 0s 5ms/step - loss: 7.3822 - accuracy: 0.6337 - val_loss: 7.5338 - val_accuracy: 0.6417
Epoch 33/50
1/9 [==>...........................] - ETA: 0s - loss: 7.3246 - accuracy: 0.6719
Epoch 33
Train-acc=0.6302
Validation-acc=0.6417
9/9 [==============================] - 0s 5ms/step - loss: 7.3793 - accuracy: 0.6302 - val_loss: 7.5319 - val_accuracy: 0.6417
Epoch 34/50
1/9 [==>...........................] - ETA: 0s - loss: 6.7741 - accuracy: 0.7344
Epoch 34
Train-acc=0.6319
Validation-acc=0.6417
9/9 [==============================] - 0s 5ms/step - loss: 7.3774 - accuracy: 0.6319 - val_loss: 7.5304 - val_accuracy: 0.6417
Epoch 35/50
1/9 [==>...........................] - ETA: 0s - loss: 8.0942 - accuracy: 0.5625
Epoch 35
Train-acc=0.6337
Validation-acc=0.6417
9/9 [==============================] - 0s 5ms/step - lo

<a id='AutoDL_사용방법'></a>
## Modeler AutoDL 사용방법
- 지금까지 스팸 계정과 정상 계정을 분류하는 MLP 모델 예제를 살펴보았습니다.
- 이번 챕터에서는 AutoDL 기능을 사용해 위에서 살펴본 MLP 모델의 하이퍼파라미터를 최적화하는 과정에 대해 다룰 것입니다.

#### 1. jupyter lab
- 먼저 autodl에서 하이퍼파라미터 튜닝을 수행할 파이썬 모델 파일을 작성합니다. 본 예제에서는 [autodl_mlp_user_classification.py](autodl_mlp_user_classification.py)을 사용합니다.
- 파이썬 모델 실행에 필요한 데이터들은 __filestorage__ 폴더에 업로드합니다.

<p align="center">
    <img src="pic/1_jupyter_lab.png" height="1000px" width="1200px">
</p>

#### 2. docker 이미지 빌드
- jupyter lab에서 작성한 파이썬 모델, filestorage의 데이터와 워크스페이스에 설치된 라이브러리를 포함한 docker 이미지를 빌드하는 단계입니다.
- Modeler > 관리 > AutoDL > 모델관리부 > 이미지 탭에서 우측 상단의 __[이미지 추가]__ 버튼을 클릭합니다.
- 이미지명, 파이썬 모델링에 사용된 딥러닝 프레임워크, 워크스페이스의 파이썬 모델 파일과 이미지 설명을 입력하고 __[빌드]__ 버튼을 클릭합니다.
- 이미지는 __BUILDING__ > __UPLOADING__ > __COMPLETE__ 순서로 빌딩됩니다.

<p align="center">
    <img src="pic/2_add_image.png" height="1000px" width="1200px">
</p>
<p align="center">
    <img src="pic/3_image_list.png" height="1000px" width="1200px">
</p>

<a id='trialtemplate_생성'></a>
#### 3. trialtemplate 생성
- autodl 실행에 필요한 trialtemplate을 생성하는 단계입니다. trialtemplate은 docker image 빌드와 동시에 자동으로 생성됩니다.
- Modeler > 관리 > AutoDL > 모델관리부 > Trial Template 탭에서 자동 생성된 trialtemplate을 확인할 수 있습니다.
- 이 단계에서 사용자는 사용할 자원의 양만 수정하면 됩니다.

<p align="center">
    <img src="pic/4_trialtemplate.png" height="1000px" width="1200px">
</p>

<a id='평가지표_수집'></a>
#### 4. AutoDL yaml 작성 및 experiment 실행
- 마지막으로, 하이퍼파라미터 튜닝에 사용되는 옵션들을 설정하는 yaml을 작성하여 AutoDL experiment를 실행해 보겠습니다.
- 옵션 관련 자세한 사항은 [상세 매뉴얼](https://accuinsight.github.io/docs/modeler/management/autodl/)에서 확인하실 수 있습니다.
- __Metrics Collector__  
  - MLP 모델 생성 및 학습 챕터에서 설정한 [평가 지표의 출력 형식](#평가지표_출력)과 관련된 옵션입니다.  
  - 출력된 평가 지표를 어떻게 수집할 것인지 설정합니다.
  - __StdOut__ 옵션은 metricsFormat에서 정의한 정규 표현식에 맞추어 평가 지표를 수집합니다.  
    metricsFormat의 디폴트 값인 ([\w|-]+)\s=\s((-?\d+)(.\d+)?)을 사용하면 __평가 지표명=평가 지표값__ 형식으로 출력된 평가 지표를 수집할 수 있습니다.
- __Algorithm__
  - 하이퍼파라미터 튜닝 알고리즘으로 __random__ 알고리즘을 선택합니다.
- __TrialCount__
  - AutoDL에서 trial은 하나의 하이퍼파라미터 조합을 적용해서 모델 성능을 평가하는 것을 의미합니다.    
    예를 들어, ParallelTrialCount=3이고 MaxTrialCount=12이면 3개의 trial을 동시에 실행하는 과정을 4번 반복하여 총 12개의 trial을 실행합니다.  
  - 본 예제에서는 trial을 1개씩 실행하여 총 10개의 trial을 실행하고 그 결과를 비교해 보겠습니다.

<p align="center">
    <img src="pic/5_1_autodl_yaml.png" height="1000px" width="1200px">
</p>

- Parameters에서 하이퍼파라미터의 타입과 탐색 범위를 지정합니다.
  - [MLP 모델 구성 및 학습](#mlp_모델_구성_및_학습) 챕터에서 설정한 하이퍼파라미터의 타입과 탐색 범위를 지정하는 단계입니다.
  - __int__, __double__: 하이퍼파라미터의 탐색 범위를 구간(FeasibleSpace)으로 지정합니다.
  - __categorical__: 하이퍼파라미터의 탐색 범위를 범주형 데이터(List)로 지정합니다.

<p align="center">
    <img src="pic/5_2_autodl_yaml.png" height="1000px" width="1200px">
</p>

- Objective에서 평가 지표를 설정합니다.  
  - 본 예제의 평가 지표는 Validation-acc이고, accuracy 값은 커지도록 학습하는 것이 목표이므로 Type으로 maximize를 선택합니다.    
  - Goal에 도달하는 순간 AutoDL experiment가 종료됩니다.  
- Trial Spec에서 trialtemplate을 선택합니다.
  - [3. trialtemplate 생성](#trialtemplate_생성)에서 생성한 autodl-mlp default.yaml을 선택합니다.  
- yaml 작성을 완료하면 __[생성]__ 버튼을 눌러 AutoDL experiment를 실행합니다.  
<p align="center">
    <img src="pic/5_3_autodl_yaml.png" height="1000px" width="1200px">
</p>

#### 5. 하이퍼파라미터 튜닝 결과
- 하이퍼파라미터 튜닝 결과를 확인합니다. autodl yaml에서 설정한 MaxTrialCount(10)만큼의 하이퍼파라미터 조합을 실험한 결과입니다.
- [하이퍼파라미터 튜닝 전](#튜닝_전_결과) 결과와 비교하여 validation accuracy가 __65%__에서 __90%__로 크게 향상된 것을 확인할 수 있습니다.

<p align="center">
    <img src="pic/6_result.png" height="1000px" width="1200px">
</p>